In [59]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from utils_all import *

import pandas as pd
import numpy as np
import os

import paramiko

# Upload or update data and Cleaning 1

In this notebook we will load all related data from MetaCentrum disctributed system and store it. 
Also in the end we will do primary cleaning of dataset (I ot of III stages)

### Update local data storage and fetch data from MetaCentrum

In [15]:
# Folder settings
DIR = '/Users/jetbrains/Yandex.Disk.localized/Diploma/DATA_DIPLOMA'
DIR_EVENT = DIR + '/data_parsed/'
DIR_ALL = DIR + '/data_all_elements/'

REMOTE_DIR = '/storage/brno2/home/shchegal/Diploma/parser'
REMOTE_DIR_EVENT = REMOTE_DIR + '/data_parsed'
REMOTE_DIR_ALL = REMOTE_DIR + '/data_all_elements_parsed'

In [16]:
def load_remote_data(remote_dir, local_dir):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect('skirit.metacentrum.cz', username='', password='')
    local_files = os.listdir(local_dir)
    sftp = ssh.open_sftp()
    dir_items = sftp.listdir_attr(remote_dir)
    for item in dir_items:
        file_name = item.filename
        if item.filename not in local_files:
            local_path = os.path.join(local_dir, file_name)
            remote_path = remote_dir + '/' + file_name
            sftp.get(remote_path, local_path)

Load data from MetaCentrum to local folder. 
2 steps: data for event components and data for all elements. Do it only if you have a new data.
It doesn't load files which already located locally

In [12]:
# Load data with event component, long operation
load_remote_data(REMOTE_DIR_EVENT, DIR_EVENT)

In [17]:
#load data with all elements, long operation
load_remote_data(REMOTE_DIR_ALL, DIR_ALL)

### Write data to the file


In [18]:
files_events = os.listdir(DIR_EVENT)[1:]
files_all = os.listdir(DIR_ALL)[1:]

Read both Events and ALL files and write into one file DATA.csv 

In [22]:
with open(DIR + "/DATA.csv", "w") as outfile:
    for f in files_events:
        with open(DIR_EVENT + f, "r") as infile:
            outfile.write(infile.read())
    for f in files_all:
        with open(DIR_ALL + f, "r") as infile:
            outfile.write(infile.read().replace("b'","'").replace("'",""))

In [23]:
DATA = pd.read_csv(DIR + '/DATA.csv', delimiter='\t')

In [24]:
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1597769 entries, 0 to 1597768
Columns: 297 entries, http://www.therapeutenfinder.com/veranstaltungen/10357-neu-psycho-holistischer-stammtisch-2015-07-12-m\u00FCnchen.html
 to auto.52
dtypes: float64(2), object(295)
memory usage: 3.5+ GB


In [25]:
DATA.shape

(1597769, 297)

In [26]:
DATA.head()

,http://www.therapeutenfinder.com/veranstaltungen/10357-neu-psycho-holistischer-stammtisch-2015-07-12-m\u00FCnchen.html,name,Neu! Psycho-Holistischer Stammtisch,52,587,52.1,288,h1,Unnamed: 8,4,...,normal.17,single,none.52,stretch.5,auto.50,auto.51,none.53,none.54,1.11,auto.52
0,https://d.lib.ncsu.edu/collections/catalog/unc...,name,NaN,0,0,0,0,meta,NaN,3,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto
1,https://d.lib.ncsu.edu/collections/catalog/unc...,startDate,1923,31,2135,19,40,time,NaN,3,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto
2,https://d.lib.ncsu.edu/collections/catalog/unc...,description,NaN,0,0,0,0,meta,NaN,3,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto
3,http://www.ticketbis.com/ar/entradas-schalke-0...,name,FC Ingolstadt - Schalke 04,26,1245,14,163,span,NaN,1,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto
4,http://www.sortiraparis.com/scenes/concert-mus...,name,Alice au pays de la jungle,14,391,21,640,h1,NaN,9,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto


Load several local files with headers

In [27]:
data_local = pd.read_csv('/Users/jetbrains/PycharmProjects/SocioPath/data/data.csv', )

In [28]:
DATA.columns = data_local.columns

In [29]:
DATA.url.unique().size

443303

In [30]:
DATA.head()

,url,meta_name,text,x_coords,y_coords,block_height,block_width,tag,num_child,num_siblings,...,white-space,-webkit-box-lines,-webkit-animation-name,-webkit-align-items,page-break-inside,-webkit-grid-auto-rows,-webkit-clip-path,max-height,stroke-width,buffered-rendering
0,https://d.lib.ncsu.edu/collections/catalog/unc...,name,NaN,0,0,0,0,meta,NaN,3,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto
1,https://d.lib.ncsu.edu/collections/catalog/unc...,startDate,1923,31,2135,19,40,time,NaN,3,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto
2,https://d.lib.ncsu.edu/collections/catalog/unc...,description,NaN,0,0,0,0,meta,NaN,3,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto
3,http://www.ticketbis.com/ar/entradas-schalke-0...,name,FC Ingolstadt - Schalke 04,26,1245,14,163,span,NaN,1,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto
4,http://www.sortiraparis.com/scenes/concert-mus...,name,Alice au pays de la jungle,14,391,21,640,h1,NaN,9,...,normal,single,none,stretch,auto,auto,none,none,1.0,auto


### Very first clean dataset 

Clean dataset from NA in URL, otherwise it doesn't fit to a memory. Also clean_df remove zero width and height elements and elements with meta tags.

In [42]:
DATA_CL = clean_df(DATA.dropna(subset=['url']))

In [51]:
DATA.shape

(1597769, 297)

In [52]:
DATA_CL.shape

(291030, 297)

In [53]:
DATA_CL.meta_name.value_counts()

not_event_element     135689
name                   63740
location               38096
startDate              33677
description            19818
auto                       4
0 fill                     1
none                       1
normal                     1
manual                     1
rgb(136, 136, 136)         1
Name: meta_name, dtype: int64

In [64]:
store = pd.HDFStore('store.h5')
store['DATA_CL'] = DATA_CL

In [55]:
%store DATA_CL

Stored 'DATA_CL' (DataFrame)
